In [216]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.figure_factory as ff

from scipy import stats

In [223]:
df = pd.read_excel("data/data_final.xlsx", engine="openpyxl")

In [224]:
sev = []
for i in range(len(df)):
    if (df["NFatal"][i] == 0 and df["NInjured"][i] == 0):
        sev.append(-1)
    elif (df["NFatal"][i] == 0 and df["NInjured"][i] > 0):
        sev.append(0)
    else:
        sev.append(1)

df["severity"] = sev

In [225]:
from scipy import stats

cat_features = ["lightConditions", "weatherConditions", "DTPType", "region", "pokritie", "road", "traffic_regime", "placeConditions", "closeObjects"]
drop_features = ["privod", "traffic_regime", "NVehicles", "date", "safety_belt", "street", "Suburb", "neispravnosti", "additionalFactor", "alco", "lightConditions", "closeObjects", "violations", "violationsAdditional"]

new_df = df.copy()

for feature in cat_features:
    new_df[feature].replace({j: i for i, j in enumerate(new_df[feature].unique())}, inplace=True)

df_dropped = df.drop(drop_features, axis=1)[(np.abs(stats.zscore(new_df.drop(drop_features, axis=1))) < 5).all(axis=1)]
# df_dropped.head()

In [226]:
# df["weatherConditions"].value_counts() # Good/bad
df_dropped["weatherConditions"].replace({"Пасмурно": "BW", "Дождь": "BW", "Снегопад": "BW", "Туман": "BW", "Ясно": "GW"}, inplace=True)

# df["pokritie"].value_counts() # Good/bad 
df_dropped["pokritie"].replace({
    "Мокрое": "BP",
    "Заснеженное": "BP",
    "Загрязненное": "BP",
    "Гололедица": "BP",
    "Со снежным накатом": "BP",
    "Сухое": "GP",
    "Обработанное противогололедными материалами": "GP"
}, inplace=True)

# df_dropped["DTPType"].value_counts()

df_dropped["DTPType"].replace({
    "Падение груза": "Car",
    "Съезд с дороги": "Car",
    "Опрокидывание": "Car",
    "Наезд на препятствие": "Car",
    "Наезд на пешехода": "Car+Human",
    "Наезд на велосипедиста": "Car+Human",
    "Падение пассажира": "Car+Human",
    "Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ": "Car+Human",
    "Столкновение": "Car+Car",
    "Наезд на стоящее ТС": "Car+Car"
}, inplace=True) # Car, Car and Car, Car and Human

df_dropped["placeConditions"].replace({
    "Выезд с прилегающей территории": "Small Speed",
    "Внутридворовая территория": "Small Speed",
    "Автостоянка (отделенная от проезжей части)": "Small Speed",
    "Автостоянка (не отделённая от проезжей части)": "Small Speed",
    "АЗС": "Small Speed",
    "Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей)": "Small Speed",
    "Мост, эстакада, путепровод": "Bridge",
    "Подход к мосту, эстакаде, путепроводу": "Bridge",
    "Тоннель": "Bridge",
    "Перегон (нет объектов на месте ДТП)": "Peregon",
    "Регулируемый перекресток": "Cross",
    "Нерегулируемый перекрёсток неравнозначных улиц (дорог)": "Cross",
    "Нерегулируемый перекрёсток равнозначных улиц (дорог)": "Cross",
    "Нерегулируемое пересечение с круговым движением": "Cross",
    "Нерегулируемый пешеходный переход": "Crosswalk",
    "Регулируемый пешеходный переход": "Crosswalk",
    "Пешеходная зона": "Crosswalk",
    "Тротуар, пешеходная дорожка": "Crosswalk",
    "Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации": "Crosswalk",
    "Регулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации": "Crosswalk",
    "Остановка общественного транспорта": "Transport",
    "Остановка трамвая": "Transport",
    "Остановка маршрутного такси": "Transport"
}, inplace=True)
df_dropped.drop(df_dropped["placeConditions"][(df_dropped["placeConditions"] == "Иное место") | (df_dropped["placeConditions"] == "Регулируемый ж/д переезд без дежурного")].index, inplace=True, axis=0)

mkad = df_dropped["road"].apply(lambda x: "МКАД" in str(x))
df_dropped.drop(mkad[mkad].index, axis=0, inplace=True)

# Эндегенность данных - камеры
# 0 - никто, 1 - пострадавшие, 2 - умершие
# {j: i for i, j in enumerate(df["region"].unique())}

# df.columns

In [227]:
# df_dropped.to_excel("data/dropped_data_nomkad.xlsx", engine="openpyxl")
# df_dropped.drop(["coordW", "coordL"], axis=1).to_excel("data/dropped_data_num.xlsx", engine="openpyxl")

In [228]:
fig = px.box(df_dropped, x="camera_count", y="severity", color='weatherConditions')
fig.show()

In [229]:
fig = px.pie(df_dropped, "pokritie", width=800, height=1000, color_discrete_sequence=px.colors.sequential.ice_r)
fig.update_layout(font=dict(family="Open Sans", size=18))
fig.update_layout({'plot_bgcolor': 'rgba(255, 255, 255, 0)', 'paper_bgcolor': 'rgba(255, 255, 255, 0)',})
fig.show()

In [267]:
fig = px.histogram(df_dropped, "severity", color_discrete_sequence=["#6948DD"])
fig.update_layout({'plot_bgcolor': 'rgba(255, 255, 255, 0)', 'paper_bgcolor': 'rgba(255, 255, 255, 0)',})
fig.update_layout(font=dict(family="Open Sans", size=18))
fig.update_layout(yaxis={"gridcolor": "#808080"}, xaxis={"gridcolor": "#808080"})
fig.update_layout({"showlegend": False})
fig.show()

In [22]:
# dff = pd.DataFrame({"pere_privod": df_dropped["pere_privod"].sum(), "poln_privod": df_dropped["poln_privod"].sum(), "zadn_privod": df_dropped["zadn_privod"].sum()}, index=[0])

# fig = px.pie(df, names=dff.columns, values=[x[0] for x in dff.T.values], width=800, height=1000, color_discrete_sequence=px.colors.sequential.ice_r)
# fig.update_layout(font=dict(family="Open Sans", size=18))
# fig.show()

In [115]:
# df_dropped["has_camera"] = df_dropped["camera_count"] > 0

# fig = px.bar(x=df_dropped.groupby("has_camera").mean().index, y=df_dropped.groupby("has_camera").mean()["severity"])
# fig.update_layout({'plot_bgcolor': 'rgba(255, 255, 255, 1)', 'paper_bgcolor': 'rgba(255, 255, 255, 1)'})
# fig.update_layout(font=dict(family="Open Sans", size=18))
# fig.update_layout(yaxis={"gridcolor": "#808080"}, xaxis={"gridcolor": "#808080"})
# fig.update_layout({"showlegend": False})
# fig.show()

In [24]:
# dens = pd.DataFrame({
#     'Южный': 13592.98 , 'Юго-Западный': 12988.54, 'Юго-Восточный': 12178.88, 'Центральный': 11777.05, 'Химки': 2338.99,
#     'Троицкий': 120.18, 'Северо-Западный': 10842.37, 'Северо-Восточный': 14060.94, 'Северный': 10429.7,
#     'Раменский': 218.96, 'Подольск': 7656.6, 'Одинцовский': 251.55, 'Новомосковский': 762.42, 'Мытыщи': 7101.56,
#     'Люберцкий': 2513.43, 'Красногорск': 1037.08, 'Зеленоградский': 6725.1, 'Западный': 9122.05,
#     'Дзержинский': 3371.46, 'Восточный': 9844.42, 'Балашиха': 2125.55, 0: 0
# }, index=[0])
# dens.drop(list(set(dens.columns) - set(df_dropped.region.unique())), axis=1, inplace=True)

# df_dropped["dens"] = dens.T[0]
# df["dens"] = dens.T[0]

In [231]:
# import json

# repl_dict = {j: i for i, j in enumerate(df_dropped["region"].unique())}

# fig = px.scatter(x=df_dropped.sort_values(by="dens")["region"].replace(repl_dict), y=df_dropped["NFatal"], trendline="ols", trendline_color_override="red", range_y=(0, 1.1))
fig = px.scatter(x=df_dropped["camera_count"], y=df_dropped["NFatal"], trendline="ols", trendline_color_override="red", range_y=(0, 3.4), range_x=(0, 8.5))
fig.update_traces(marker=dict(size=10, opacity=.05))
fig.update_layout({'plot_bgcolor': 'rgba(255, 255, 255, 0)', 'paper_bgcolor': 'rgba(255, 255, 255, 0)'})
fig.update_layout(font=dict(family="Open Sans", size=18))
fig.update_layout(yaxis={"gridcolor": "#808080"}, xaxis={"gridcolor": "#808080"})
fig.update_xaxes(showline=True, linewidth=2, linecolor="#808080")
fig.update_yaxes(showline=True, linewidth=2, linecolor="#808080")
fig.update_layout({"showlegend": False})
fig.show()

results = px.get_trendline_results(fig)
print(results.px_fit_results[0].params)

[ 0.03687832 -0.00300732]


In [88]:
# import statsmodels.api as sm

# # X2 = sm.add_constant(df_dropped["NInjured"])
# # est = sm.OLS(df_dropped.drop(set(df_dropped.columns) - set(["camera_count"]), axis=1), X2)
# # est2 = est.fit()
# # print(est2.summary())

# X2 = sm.add_constant(pd.get_dummies(df_dropped.drop(set(df_dropped.columns) - set(["region"]), axis=1)).join(df_dropped["camera_count"]), prepend=False)
# est = sm.OLS(df_dropped["NFatal"], X2).fit()
# est.summary()

In [119]:
# px.bar(df_dropped, "region")

In [117]:
# fig = px.bar(x=dens.T.sort_values(by=0).T.columns, y=(dens.T[0] / 10000 * -1).sort_values(ascending=False))
# fig.add_bar(x=df_dropped.groupby("region").median().index, y=df_dropped.groupby("region").median()["severity"], marker=dict(color="#6948DD"))
# fig.update_layout({'plot_bgcolor': 'rgba(255, 255, 255, 2)', 'paper_bgcolor': 'rgba(255, 255, 255, 2)'})
# fig.update_layout(yaxis={'visible': False, 'showticklabels': False})
# fig.update_layout(font=dict(family="Open Sans", size=18))
# fig.update_layout({"showlegend": False})
# fig.show()

In [29]:
mean = df_dropped["camera_count"].mean()
median = df_dropped["camera_count"].median()
stdev_pluss = np.std(df_dropped["camera_count"].values)
stdev_minus = np.std(df_dropped["camera_count"].values) * -1

fig = ff.create_distplot([df_dropped["camera_count"].values], ["camera_count"])
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.add_shape(type="line",x0=mean, x1=mean, y0=0, y1=0.4, xref='x', yref='y', line=dict(color='blue', dash='dash'))
fig.add_shape(type="line",x0=median, x1=median, y0=0, y1=0.4, xref='x', yref='y', line=dict(color='purple', dash='dash'))
fig.add_shape(type="line",x0=stdev_pluss, x1=stdev_pluss, y0=0, y1=0.4, xref='x', yref='y', line=dict(color='red', dash='dash'))
fig.add_shape(type="line",x0=stdev_minus, x1=stdev_minus, y0=0, y1=0.4, xref='x', yref='y', line=dict(color='red', dash='dash'))
fig.update_layout(font=dict(family="Open Sans", size=18))
fig.update_layout({"showlegend": False})
fig.update_layout(yaxis={"gridcolor": "#808080"}, xaxis={"gridcolor": "#808080"})
fig.show()

In [95]:
mean = df_dropped["NInjured"].mean()
median = df_dropped["NInjured"].median()
stdev_pluss = np.std(df_dropped["NInjured"].values)
stdev_minus = np.std(df_dropped["NInjured"].values) * -1

fig = ff.create_distplot([df_dropped["NInjured"].values], ["NInjured"], colors=["#6948DD"])
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.add_shape(type="line",x0=mean, x1=mean, y0=0, y1=4, xref='x', yref='y', line=dict(color='blue', dash='dash'))
fig.add_shape(type="line",x0=median, x1=median, y0=0, y1=4, xref='x', yref='y', line=dict(color='purple', dash='dash'))
fig.add_shape(type="line",x0=stdev_pluss, x1=stdev_pluss, y0=0, y1=4, xref='x', yref='y', line=dict(color='red', dash='dash'))
fig.add_shape(type="line",x0=stdev_minus, x1=stdev_minus, y0=0, y1=4, xref='x', yref='y', line=dict(color='red', dash='dash'))
fig.update_layout(font=dict(family="Open Sans", size=18))
fig.update_layout({"showlegend": False})
fig.update_layout(yaxis={"gridcolor": "#808080"}, xaxis={"gridcolor": "#808080"})
fig.show()

In [52]:
fig = ff.create_2d_density(x=df_dropped["NFatal"], y=df_dropped["NInjured"], hist_color="#6948DD", colorscale=px.colors.sequential.ice)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.update_layout(font=dict(family="Open Sans", size=18))
fig.update_layout({"showlegend": False})
fig.update_layout(yaxis={"gridcolor": "#808080"}, xaxis={"gridcolor": "#808080"})
fig.show()

In [199]:
fig = px.density_mapbox(df_dropped, lat="coordW", lon="coordL", radius=3, mapbox_style="stamen-terrain", height=1000)
fig.show()

In [169]:
with open("data/camera.txt", "r") as camera_file:
    cameras = pd.DataFrame([list(map(float, x.strip().split())) for x in camera_file.readlines()])

fig = px.density_mapbox(cameras, lat=0, lon=1, radius=5, mapbox_style="open-street-map", height=1000)
fig.show()

In [127]:
df_old = pd.read_excel("data/Аварии - датасет.xlsx", engine="openpyxl")

In [141]:
# fig = px.density_mapbox(df_old, lat="coordW", lon="coordL", radius=3, mapbox_style="stamen-terrain", height=800, width=800)
# fig = px.density_mapbox(df_dropped, lat="coordW", lon="coordL", radius=3, mapbox_style="stamen-terrain", height=800, width=800)
# fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
# fig.update_layout(font=dict(family="Open Sans", size=18))
# fig.update_layout({"showlegend": False})
# fig.update_layout(yaxis={"gridcolor": "#808080"}, xaxis={"gridcolor": "#808080"})
# fig.show()